# COVID-19 Overview
> Tracking coronavirus total cases, deaths and new cases by country. Additionally, a detailed view is provided for the US(by state), Europe

- comments: true
- author: Pratap Vardhan
- categories: [overview, interactive]
- image: images/covid-overview.png
- permalink: /covid-overview/
- hide: false

In [1]:
#hide
print('''
Example of using jupyter notebook, pandas (data transformations), jinja2 (html, visual)
to create visual dashboards with fastpages
You see also the live version on https://gramener.com/enumter/covid19/
''')


Example of using jupyter notebook, pandas (data transformations), jinja2 (html, visual)
to create visual dashboards with fastpages
You see also the live version on https://gramener.com/enumter/covid19/



In [2]:
#hide
import numpy as np
import pandas as pd
from jinja2 import Template
from IPython.display import HTML

In [3]:
#hide

# FETCH
base_url = 'https://raw.githubusercontent.com/pratapvardhan/notebooks/master/covid19/'
paths = {
    'mapping': base_url + 'mapping_countries.csv',
    'overview': base_url + 'overview.tpl'
}

def get_mappings(url):
    df = pd.read_csv(url)
    return {
        'df': df,
        'replace.country': dict(df.dropna(subset=['Name']).set_index('Country')['Name']),
        'map.continent': dict(df.set_index('Name')['Continent'])
    }

mapping = get_mappings(paths['mapping'])

def get_template(path):
    from urllib.parse import urlparse
    if bool(urlparse(path).netloc):
        from urllib.request import urlopen
        return urlopen(path).read().decode('utf8')
    return open(path).read()

def get_frame(name):
    url = (
        'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/'
        f'csse_covid_19_time_series/time_series_covid19_{name}_global.csv')
    df = pd.read_csv(url)
    # rename countries
    df['Country/Region'] = df['Country/Region'].replace(mapping['replace.country'])
    return df

def get_dates(df):
    dt_cols = df.columns[~df.columns.isin(['Province/State', 'Country/Region', 'Lat', 'Long'])]
    LAST_DATE_I = -1
    # sometimes last column may be empty, then go backwards
    for i in range(-1, -len(dt_cols), -1):
        if not df[dt_cols[i]].fillna(0).eq(0).all():
            LAST_DATE_I = i
            break
    return LAST_DATE_I, dt_cols

In [4]:
#hide
COL_REGION = 'Country/Region'
# Confirmed, Recovered, Deaths
df = get_frame('confirmed')
# dft_: timeseries, dfc_: today country agg
dft_cases = df
dft_deaths = get_frame('deaths')
dft_recovered = get_frame('recovered')
LAST_DATE_I, dt_cols = get_dates(df)

dt_today = dt_cols[LAST_DATE_I]
dt_5ago = dt_cols[LAST_DATE_I-5]


dfc_cases = dft_cases.groupby(COL_REGION)[dt_today].sum()
dfc_deaths = dft_deaths.groupby(COL_REGION)[dt_today].sum()
dfp_cases = dft_cases.groupby(COL_REGION)[dt_5ago].sum()
dfp_deaths = dft_deaths.groupby(COL_REGION)[dt_5ago].sum()

In [5]:
#hide
df_table = (pd.DataFrame(dict(Cases=dfc_cases, Deaths=dfc_deaths, PCases=dfp_cases, PDeaths=dfp_deaths))
             .sort_values(by=['Cases', 'Deaths'], ascending=[False, False])
             .reset_index())
df_table.rename(columns={'index': 'Country/Region'}, inplace=True)
for c in 'Cases, Deaths'.split(', '):
    df_table[f'{c} (+)'] = (df_table[c] - df_table[f'P{c}']).clip(0)  # DATA BUG
df_table['Fatality Rate'] = (100 * df_table['Deaths'] / df_table['Cases']).round(1)
df_table['Continent'] = df_table['Country/Region'].map(mapping['map.continent'])
df_table.head(15)

Country/Region     Cases  Deaths    PCases  PDeaths  Cases (+)  \
0               US  30706126  555001  30393805   551030     312321   
1           Brazil  12984956  331433  12658109   317646     326847   
2            India  12589067  165101  12149335   162468     439732   
3           France   4883174   96808   4646014    95495     237160   
4           Russia   4529576   98713   4486078    96817      43498   
5   United Kingdom   4373798  127078   4355867   126912      17931   
6            Italy   3668264  111030   3561012   108879     107252   
7           Turkey   3487050   32263   3277880    31385     209170   
8            Spain   3300965   75698   3275819    75305      25146   
9          Germany   2896329   77060   2818630    76389      77699   
10        Colombia   2446219   64094   2397731    63255      48488   
11          Poland   2438542   54941   2288826    52392     149716   
12       Argentina   2393492   56199   2332765    55736      60727   
13          Mexico   2250458  204147   2232910   202633      17548   
14            Iran   1932074   63160   1875234    62569      56840   

    Deaths (+)  Fatality Rate      Continent  
0         3971            1.8  North America  
1        13787            2.6  South America  
2         2633            1.3           Asia  
3         1313            2.0         Europe  
4         1896            2.2         Europe  
5          166            2.9         Europe  
6         2151            3.0         Europe  
7          878            0.9           Asia  
8          393            2.3         Europe  
9          671            2.7         Europe  
10         839            2.6  South America  
11        2549            2.3         Europe  
12         463            2.3  South America  
13        1514            9.1  North America  
14         591            3.3           Asia

In [6]:
#hide
#delete problematic countries from table
df_table = df_table[~df_table['Country/Region'].isin(['Cape Verde', 'Cruise Ship', 'Kosovo'])]

In [7]:
#hide
# world, china, europe, us
metrics = ['Cases', 'Deaths', 'Cases (+)', 'Deaths (+)']
s_china = df_table[df_table['Country/Region'].eq('China')][metrics].sum().add_prefix('China ')
s_us = df_table[df_table['Country/Region'].eq('US')][metrics].sum().add_prefix('US ')
s_eu = df_table[df_table['Continent'].eq('Europe')][metrics].sum().add_prefix('EU ')
summary = {'updated': pd.to_datetime(dt_today), 'since': pd.to_datetime(dt_5ago)}
summary = {**summary, **df_table[metrics].sum(), **s_china, **s_us, **s_eu}
summary

{'updated': Timestamp('2021-04-04 00:00:00'),
 'since': Timestamp('2021-03-30 00:00:00'),
 'Cases': 131230484,
 'Deaths': 2851095,
 'Cases (+)': 3106682,
 'Deaths (+)': 49530,
 'China Cases': 101861,
 'China Deaths': 4841,
 'China Cases (+)': 151,
 'China Deaths (+)': 0,
 'US Cases': 30706126,
 'US Deaths': 555001,
 'US Cases (+)': 312321,
 'US Deaths (+)': 3971,
 'EU Cases': 40437264,
 'EU Deaths': 924776,
 'EU Cases (+)': 1070334,
 'EU Deaths (+)': 16970}

In [8]:
#hide
dft_ct_cases = dft_cases.groupby(COL_REGION)[dt_cols].sum()
dft_ct_new_cases = dft_ct_cases.diff(axis=1).fillna(0).astype(int)
dft_ct_new_cases.head()

1/22/20  1/23/20  1/24/20  1/25/20  1/26/20  1/27/20  1/28/20  \
Country/Region                                                                  
Afghanistan           0        0        0        0        0        0        0   
Albania               0        0        0        0        0        0        0   
Algeria               0        0        0        0        0        0        0   
Andorra               0        0        0        0        0        0        0   
Angola                0        0        0        0        0        0        0   

                1/29/20  1/30/20  1/31/20  ...  3/26/21  3/27/21  3/28/21  \
Country/Region                             ...                              
Afghanistan           0        0        0  ...       28       36        4   
Albania               0        0        0  ...      449      425      493   
Algeria               0        0        0  ...      114       93       86   
Andorra               0        0        0  ...       45       77       41   
Angola                0        0        0  ...       47       70       32   

                3/29/21  3/30/21  3/31/21  4/1/21  4/2/21  4/3/21  4/4/21  
Country/Region                                                             
Afghanistan          28       62       70      63      55      23      81  
Albania             285      304      434     349     336     341     348  
Algeria             110      115      131     112     125      95      98  
Andorra              38       56       66      43      62      59      57  
Angola               69       50      129      88      68     112      52  

[5 rows x 439 columns]

In [9]:
#hide_input
template = Template(get_template(paths['overview']))
html = template.render(
    D=summary, table=df_table,  # REMOVE .head(20) to see all values
    newcases=dft_ct_new_cases.loc[:, dt_cols[LAST_DATE_I-50]:dt_cols[LAST_DATE_I]],
    np=np, pd=pd, enumerate=enumerate)
HTML(f'<div>{html}</div>')

Visualizations by [Pratap Vardhan](https://twitter.com/PratapVardhan)[^1]

[^1]: Source: ["COVID-19 Data Repository by Johns Hopkins CSSE"](https://systems.jhu.edu/research/public-health/ncov/) [GitHub repository](https://github.com/CSSEGISandData/COVID-19). Link to [notebook](https://github.com/pratapvardhan/notebooks/blob/master/covid19/covid19-overview.ipynb), [orignal interactive](https://gramener.com/enumter/covid19/)